In [1]:
import numpy as np
import scipy.io.wavfile as wav
from speechpy.feature import mfcc
import tensorflow as tf

2023-10-15 03:11:24.579891: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-15 03:11:24.637735: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-15 03:11:24.638555: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 03:11:25.801037: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
mean_signal_length = 88500

In [4]:
def get_feature_vector_from_mfcc(file_path: str, flatten: bool, mfcc_len: int = 39) -> np.ndarray:
    fs, signal = wav.read(file_path)
    s_len = len(signal)
    # pad the signals to have same size if lesser than required
    # else slice them
    if s_len < mean_signal_length:
        pad_len = mean_signal_length - s_len
        pad_rem = pad_len % 2
        pad_len //= 2
        signal = np.pad(signal, (pad_len, pad_len + pad_rem),
                        'constant', constant_values=0)
    else:
        pad_len = s_len - mean_signal_length
        pad_len //= 2
        signal = signal[pad_len:pad_len + mean_signal_length]
    mel_coefficients = mfcc(signal, fs, num_cepstral=mfcc_len)
    if flatten:
        # Flatten the data
        mel_coefficients = np.ravel(mel_coefficients)
    return mel_coefficients

In [15]:
filename = './voice_database/test2/1.wav'
features = get_feature_vector_from_mfcc(filename, flatten=False)
features = np.expand_dims(features, axis=2)

In [13]:
labels = ["Neutral", "Angry", "Happy", "Sad"]

In [8]:
model = tf.keras.models.load_model('./sentiment_models/sentiment_classifier.h5')

2023-10-15 03:17:59.451229: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [16]:
pred = np.argmax(model.predict(np.array([features])))
pred_label = labels[pred]
pred_label

1/1 [==============================] - 0s 19ms/step


'Sad'